In [1]:
import utils

import tensorflow as tf

In [2]:
from tensorflow.keras.models import load_model 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)

In [3]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [4]:
data_dir = 'input'
utils.download_extract('celeba', data_dir)

Extracting celeba...


In [ ]:
def discriminator(images, alpha = 0.2, dropout = 0.8):
    """
    Create the discriminator network
    :param images: Tensor of input image(s)
    :param alpha: The leakage amount from the relus.
    :param dropout: How much dropout we want for trainning the discriminator
    :return: Tuple of (tensor output of the discriminator, tensor logits of the discriminator)
    """
    
    h1 = tf.keras.layers.conv2d(images, 64, 5, strides = 2, padding = 'same')
    h1 = tf.maximum(alpha * h1, h1)

    h2 = tf.keras.layers.conv2d(h1, 128, 5, strides=2, padding='same')
    h2 = tf.keras.layers.batch_normalization(h2, training=True)
    h2 = tf.maximum(alpha * h2, h2)

    h3 = tf.keras.layers.conv2d(h2, 256, 5, strides=2, padding='same')
    h3 = tf.keras.layers.batch_normalization(h3, training=True)
    h3 = tf.maximum(alpha * h3, h3)

    flat = tf.keras.layers.Reshape(h3, (-1, 4*4*256))
    dropout_layer = tf.layers.dropout(inputs=flat, rate=dropout)
    logits = tf.layers.dense(dropout_layer, 1)
    out = tf.sigmoid(logits)
        
    return out, logits